In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/17 16:46:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Job1

In [165]:
rdp = sc.parallelize([(("1999", "Z"), "ciao"), (("2000", "SB"), "r"), (("1999","Z"), "ciao"), (("1999","Z"), "aaaa"), (("1999","A"), "ciao"), (("2000", "A"), "miao"), (("1999", "BB"), "miao"), (("2000", "A"), "miao"), (("2000", "A"), "ciao"), (("2000", "A"), "ciao")])
rdp.collect()

[(('1999', 'Z'), 'ciao'),
 (('2000', 'SB'), 'r'),
 (('1999', 'Z'), 'ciao'),
 (('1999', 'Z'), 'aaaa'),
 (('1999', 'A'), 'ciao'),
 (('2000', 'A'), 'miao'),
 (('1999', 'BB'), 'miao'),
 (('2000', 'A'), 'miao'),
 (('2000', 'A'), 'ciao'),
 (('2000', 'A'), 'ciao')]

In [166]:
list(rdp.countByKey().items())

[(('1999', 'Z'), 3),
 (('2000', 'SB'), 1),
 (('1999', 'A'), 1),
 (('2000', 'A'), 4),
 (('1999', 'BB'), 1)]

In [167]:
u2 = sc.parallelize(list(rdp.countByKey().items()))

In [168]:
rdp_joined = rdp.join(u2)
rdp_joined.collect()

[(('1999', 'A'), ('ciao', 1)),
 (('2000', 'A'), ('miao', 4)),
 (('2000', 'A'), ('miao', 4)),
 (('2000', 'A'), ('ciao', 4)),
 (('2000', 'A'), ('ciao', 4)),
 (('1999', 'BB'), ('miao', 1)),
 (('1999', 'Z'), ('ciao', 3)),
 (('1999', 'Z'), ('ciao', 3)),
 (('1999', 'Z'), ('aaaa', 3)),
 (('2000', 'SB'), ('r', 1))]

In [169]:
out = rdp_joined.reduceByKey(lambda x,y: (x[0]+" "+y[0],x[1]))

In [170]:
out.collect()

[(('1999', 'A'), ('ciao', 1)),
 (('2000', 'A'), ('miao miao ciao ciao', 4)),
 (('1999', 'BB'), ('miao', 1)),
 (('1999', 'Z'), ('ciao ciao aaaa', 3)),
 (('2000', 'SB'), ('r', 1))]

In [171]:
out_to_sort = out.map(lambda x: (x[0][0],(x[0][1], x[1][1], x[1][0])))

In [172]:
out_to_sort.collect()

[('1999', ('A', 1, 'ciao')),
 ('2000', ('A', 4, 'miao miao ciao ciao')),
 ('1999', ('BB', 1, 'miao')),
 ('1999', ('Z', 3, 'ciao ciao aaaa')),
 ('2000', ('SB', 1, 'r'))]

In [178]:
out_sorted = out_to_sort.groupByKey().mapValues(lambda x: sorted(x, key=lambda y: y[1], reverse=True)[:2])

In [179]:
out_sorted.collect()

[('2000', [('A', 4, 'miao miao ciao ciao'), ('SB', 1, 'r')]),
 ('1999', [('Z', 3, 'ciao ciao aaaa'), ('A', 1, 'ciao')])]

In [190]:
def a(line):
    out = []
    for tupla in line[1]:
        for word in tupla[2].split():
            if(len(word) >= 4):
                out.append(((line[0],tupla[0]),word))
    return out

out_word = out_sorted.flatMap(a)

In [191]:
out_word.collect()

[(('2000', 'A'), 'miao'),
 (('2000', 'A'), 'miao'),
 (('2000', 'A'), 'ciao'),
 (('2000', 'A'), 'ciao'),
 (('1999', 'Z'), 'ciao'),
 (('1999', 'Z'), 'ciao'),
 (('1999', 'Z'), 'aaaa'),
 (('1999', 'A'), 'ciao')]

In [200]:
out_word_counted = list(out_word.countByValue().items())
out_word_counted

[((('2000', 'A'), 'miao'), 2),
 ((('2000', 'A'), 'ciao'), 2),
 ((('1999', 'Z'), 'ciao'), 2),
 ((('1999', 'Z'), 'aaaa'), 1),
 ((('1999', 'A'), 'ciao'), 1)]

In [202]:
out_word_counted_rdd = sc.parallelize(out_word_counted)
out_word_counted_rdd.collect()

[((('2000', 'A'), 'miao'), 2),
 ((('2000', 'A'), 'ciao'), 2),
 ((('1999', 'Z'), 'ciao'), 2),
 ((('1999', 'Z'), 'aaaa'), 1),
 ((('1999', 'A'), 'ciao'), 1)]

In [205]:
final_out_to_order = out_word_counted_rdd.map(lambda x: (x[0][0],(x[0][1],x[1])))
final_out_to_order.collect()

[(('2000', 'A'), ('miao', 2)),
 (('2000', 'A'), ('ciao', 2)),
 (('1999', 'Z'), ('ciao', 2)),
 (('1999', 'Z'), ('aaaa', 1)),
 (('1999', 'A'), ('ciao', 1))]

In [214]:
final_out_to_order.groupByKey().mapValues(lambda x: sorted(x, key=lambda y: y[1], reverse=True)[:2]).collect()

[(('1999', 'A'), [('ciao', 1)]),
 (('2000', 'A'), [('miao', 2), ('ciao', 2)]),
 (('1999', 'Z'), [('ciao', 2), ('aaaa', 1)])]

# Unione

In [174]:
content = ["Id","ProductId", "1", "AA"]
c1 = [30, 3, 7, 13]
c2 = [1,2,3,4]

content_rdd = sc.parallelize([content])
c1_rdd = sc.parallelize([c1])
c2_rdd = sc.parallelize([c2])

In [175]:
content_rdd.union(c1_rdd).union(c2_rdd).collect()

[['Id', 'ProductId', '1', 'AA'], [30, 3, 7, 13], [1, 2, 3, 4]]